# 15 mar 2018

O objetivo inicial era buscar dados sobre proposições disponíveis [aqui](https://dadosabertos.camara.leg.br/api/v2/proposicoes/). Porém, como nosso plano é construir um MVP pra apresentar semana que vem e vasculhar esses dados dará um trabalho enorme (quebrando, assim, o fluxo de trabalho), decidi começar logo o planejamento pra construção do MVP.

Precisaremos, basicamente dos seguintes passos:
  - Analisar o dataset montado (_2018-02-15-tse-candidates.xz_) e descobrir quais são as informações importantes
  - Modelar as primeiras tabelas do banco de dados
  - Construir um endpoint com node/express pra servir como api
  - Construir uma página web rústica para ser disponibilizada ao público (JS/react)


## Análise do Dataset

Informações disponíveis no dataset
```
  DATA_GERACAO
  HORA_GERACAO
  ANO_ELEICAO
  NUM_TURNO
  DESCRICAO_ELEICAO
  SIGLA_UF
  SIGLA_UE
  DESCRICAO_UE
  CODIGO_CARGO
  DESCRICAO_CARGO
  NOME_CANDIDATO
  SEQUENCIAL_CANDIDATO
  NUMERO_CANDIDATO
  CPF_CANDIDATO
  NOME_URNA_CANDIDATO
  COD_SITUACAO_CANDIDATURA
  DES_SITUACAO_CANDIDATURA
  NUMERO_PARTIDO
  SIGLA_PARTIDO
  NOME_PARTIDO
  CODIGO_LEGENDA
  SIGLA_LEGENDA
  COMPOSICAO_LEGENDA
  NOME_LEGENDA
  CODIGO_OCUPACAO
  DESCRICAO_OCUPACAO
  DATA_NASCIMENTO
  NUM_TITULO_ELEITORAL_CANDIDATO
  IDADE_DATA_ELEICAO
  CODIGO_SEXO
  DESCRICAO_SEXO
  COD_GRAU_INSTRUCAO
  DESCRICAO_GRAU_INSTRUCAO
  CODIGO_ESTADO_CIVIL
  DESCRICAO_ESTADO_CIVIL
  CODIGO_NACIONALIDADE
  DESCRICAO_NACIONALIDADE
  SIGLA_UF_NASCIMENTO
  CODIGO_MUNICIPIO_NASCIMENTO
  NOME_MUNICIPIO_NASCIMENTO
  DESPESA_MAX_CAMPANHA
  COD_SIT_TOT_TURNO
  DESC_SIT_TOT_TURNO
  NM_EMAIL
```


## Modelagem das Tabelas

Rascunhando uma maneira de agrupar as informações

```
ANO_ELEICAO
NUM_TURNO
DESCRICAO_ELEICAO
SIGLA_UF

SIGLA_UE => se for número - é município (código TSE do município) | se for letra é estado | BR para presidente
DESCRICAO_UE

NUMERO_CANDIDATO
NOME_URNA_CANDIDATO
NUMERO_PARTIDO
SIGLA_PARTIDO
NOME_PARTIDO
NOME_LEGENDA (pode ser nulo)
DESPESA_MAX_CAMPANHA
COD_SIT_TOT_TURNO
DESC_SIT_TOT_TURNO

CODIGO_CARGO
DESCRICAO_CARGO

COD_SITUACAO_CANDIDATURA
DES_SITUACAO_CANDIDATURA

CODIGO_OCUPACAO
DESCRICAO_OCUPACAO

CODIGO_SEXO (0 = não informado, 2 = masculino, 4 = feminino)
DESCRICAO_SEXO

COD_GRAU_INSTRUCAO
DESCRICAO_GRAU_INSTRUCAO

CODIGO_ESTADO_CIVIL
DESCRICAO_ESTADO_CIVIL

CODIGO_NACIONALIDADE ?
DESCRICAO_NACIONALIDADE ?

NOME_CANDIDATO
DATA_NASCIMENTO
CPF_CANDIDATO
NUM_TITULO_ELEITORAL_CANDIDATO ?
SIGLA_UF_NASCIMENTO
NM_EMAIL

CODIGO_MUNICIPIO_NASCIMENTO - não funciona!
NOME_MUNICIPIO_NASCIMENTO
```

### Tabelas

_Obs1.: Raça e sexo foram ignorados por acreditarmos que essas características devem ser irrelevantes para a escolha de um representante público_

_Obs2.: Decidimos pela nomenclatura das tabelas no singular por [isso](https://stackoverflow.com/a/5841297)_

_Obs3.: abc_

**Candidatos**
- nome (NOME_CANDIDATO)
- data_nascimento (DATA_NASCIMENTO)
- cpf (CPF_CANDIDATO)
- titulo_eleitoral (NUM_TITULO_ELEITORAL_CANDIDATO)
- email (NM_EMAIL)
- id_cidade_natal
- id_nacionalidade
- id_grau_instrucao
- id_ocupacao

**Cidades**
- id
- codigo_tse (SIGLA_UE)
- nome (DESCRICAO_UE | NOME_MUNICIPIO_NASCIMENTO)
- id_estado

**Estados**
- id
- sigla
- nome

**Nacionalidades**
- id
- codigo_tse (CODIGO_NACIONALIDADE)
- nome (DESCRICAO_NACIONALIDADE)

**Graus de Instrução**
- id | codigo_tse (COD_GRAU_INSTRUCAO)
- descricao (DESCRICAO_GRAU_INSTRUCAO)

**Ocupações**
- id | codigo_tse (CODIGO_OCUPACAO)
- descricao (DESCRICAO_OCUPACAO)

**Candidaturas**
- id
- id_candidato
- id_eleicao
- id_cidade (SIGLA_UE)
- id_estado (SIGLA_UF)
- numero_candidato (NUMERO_CANDIDATO)
- nome_urna (NOME_URNA_CANDIDATO)
- id_partido
- legenda (NOME_LEGENDA)
- id_cargo
- despesa_maxima (DESPESA_MAX_CAMPANHA)
- id_situacao
- id_resultado

**Eleições**
- id
- ano (ANO_ELEICAO)
- descricao (DESCRICAO_ELEICAO)

**Partidos**
- id
- numero (NUMERO_PARTIDO)
- sigla (SIGLA_PARTIDO)
- nome (NOME_PARTIDO)

**Cargos**
- id | codigo_tse (CODIGO_CARGO)
- nome

**Resultado da Candidatura**
- id | codigo_tse (COD_SITUACAO_CANDIDATURA)
- desceicao (DES_SITUACAO_CANDIDATURA)

**Resultado da Candidatura**
- id | codigo_tse (COD_SIT_TOT_TURNO)
- desceicao (DESC_SIT_TOT_TURNO)

### Alguns problemas relacionados ao Dataset

Durante o trabalho pude perceber que algumas informações estão meio bagunçadas, como por exemplo, o valor do _código_ estar trocada com o valor da _descrição_:

| COD_SIT_TOT_TURNO  | DESC_SIT_TOT_TURNO                           |
| ------------------ | -------------------------------------------- |
| NaN                | NaN                                          |
| -3                 | #NULO#                                       |
| NÃO ELEITO         | NaN                                          |
| SUPLENTE           | NaN                                          |
| -1                 | #NULO#                                       |
| 1                  | ELEITO                                       |
| 2                  | ELEITO POR QP                                |
| 3                  | ELEITO POR MÉDIA                             |
| 4                  | NÃO ELEITO                                   |
| 5                  | SUPLENTE                                     |
| 6                  | 2º TURNO                                     |
| 7                  | RENÚNCIA/FALECIMENTO/CASSAÇÃO APÓS A ELEIÇÃO |
| 8                  | REGISTRO NEGADO ANTES DA ELEIÇÃO             |
| 9                  | REGISTRO NEGADO APÓS A ELEIÇÃO               |
| 10                 | SUBSTITUÍDO                                  |
| 11                 | INDEFERIDO COM RECURSO                       |
| 12                 | CASSADO COM RECURSO                          |

`Usando tabelas com markdown: https://help.github.com/articles/organizing-information-with-tables/`

NOME_CANDIDATO 	ANO_ELEICAO 	SIGLA_UE 	DESCRICAO_UE 	NUM_TURNO 	DESCRICAO_CARGO 	DESC_SIT_TOT_TURNO
743740 	SONIA REGINA SILVA MEREU 	2008 	62499 	BOTUCATU 	1 	VEREADOR 	NaN
NOME_CANDIDATO 	ANO_ELEICAO 	SIGLA_UE 	DESCRICAO_UE 	NUM_TURNO 	DESCRICAO_CARGO 	DESC_SIT_TOT_TURNO
653018 	DAVID MARCELINO RIBEIRO 	2008 	76775 	MANDAGUARI 	1 	VEREADOR 	NaN

Será preciso analisar todas as duplas (código/descrição) para identificar os padrões antes de rodar os SQLs